In [ ]:
import networkx as nx
from networkx import DiGraph
import random
import matplotlib.pyplot as plt
from collections import Counter
from model import Move, Board, ALL_MOVES, draw_graph, RandomPlayer, MiniMaxPlayer

In [ ]:
class Game():
    def __init__(self, player1, player2):
        self.player1 = player1
        self.player2 = player2
        self._active_player = player1
        self._inactive_player = player2
        self._turns = 1
        self.player1.idx = 1 # test that these can be switched without affecting game score
        self.player2.idx = 0
        
    @property
    def active_player(self):
        return self._active_player
    
    @property
    def inactive_player(self):
        return self._inactive_player
    
    @property
    def turns(self):
        return self._turns
    
    def end_of_turn(self):
        '''Switch players and increment turn count'''
        self._active_player, self._inactive_player = self._inactive_player, self._active_player
        self._turns += 1

In [ ]:
results = []

for n_games in range(50):

    # Set up new game
    board = Board((1,1), (1,1))
    player1 = RandomPlayer(name='random')
    player2 = MiniMaxPlayer(name='minimax', max_depth=30) # minimax idx=0
    game = Game(player1, player2) 

    while not board.is_winner:
        move = game.active_player.select_move(board)
        board = board.update_board(move)

        if board.is_winner:
            print('Winner:', game.active_player.name)
            results.append(game.active_player.name)
            break
        
        # End of turn, swithc players etc.
        game.end_of_turn()
        if game.turns > 30:
            break
    
    

In [ ]:
Counter(results)